In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from numpy.testing import assert_array_almost_equal
import GPy
import gp_grief
import gp_grief.grid
from pdb import set_trace
from time import time
gp_grief.debug()

In [2]:
def rastrigin(x,lin_term=None):
    """
    Rastrigin test function
    input should be in range [-5.12, 5.12]
    if x in range [0,1], can transform by rastrigin((x*2-1)*5.12)
    if lin_term is not None then will add a linear term to the first dimension. This helps
    to make the function non-symetric wrt the input dimensions
    """
    assert x.ndim == 2
    d = x.shape[1]
    f = 10*d
    for i in range(d):
        f = f+(np.power(x[:,i,None],2) - 10*np.cos(2*np.pi*x[:,i,None]));
    if lin_term is not None:
        f += lin_term*x[:,(0,)]
    return f

# `gp_grief.models.GPRegression` Testing
Will compare with GPy across various problems.

Note that this assumes the model defaults are the same as those for GPy.

## Sweep accross various Problem Dimensions 
And consider non-default active dimensions

In [3]:
for d in [1,10]:
    for active_dims in [None,]:#, np.int32(np.floor(0.5*d))]:
        print '\n' + '*' * 80
        print 'd = %d, active_dims = %s' % (d, repr(active_dims))

        # generate data
        np.random.seed(0)
        N = 100
        x = np.random.uniform(size=(N,d)) # generate dataset
        y = rastrigin((x*2-1)*5.12, lin_term=1.)
        xx = np.random.uniform(size=(N+1,d)) # generate test set

        # initialize GPy.models.GPRegression
        if active_dims is None:
            kern = GPy.kern.RBF(d)
            mg = GPy.models.GPRegression(x,y,kern)
        else: # I want to consider only a subset of the dimensions
            kern = GPy.kern.RBF(np.size(active_dims))
            x_reduced = x[:,active_dims].reshape((-1,np.size(active_dims)))
            mg = GPy.models.GPRegression(x_reduced, y, kern)
        
        # initialize gp_grief model
        kern = gp_grief.kern.RBF(d, active_dims=active_dims)
        m = gp_grief.models.GPRegression(x,y,kern)
        
        print 'checking gradient'
        m.checkgrad() 
        
        print 'checking initial log-likelihood'
        assert_array_almost_equal(mg.log_likelihood(), m.log_likelihood(), decimal=3) 
        
        print 'testing if alpha matches'
        m.fit()
        alpha_gpy = mg.posterior.woodbury_vector
        assert_array_almost_equal(alpha_gpy, m._alpha, decimal=3)
        
        print 'testing if predictions match...'
        yyh = m.predict(xx, compute_var='full')
        yyh_gpy = mg.predict(xx,full_cov=True)
        print '... mean'
        assert_array_almost_equal(yyh_gpy[0], yyh[0], decimal=3)
        print '... variance'
        assert_array_almost_equal(yyh_gpy[1], yyh[1], decimal=2)
        
        print 'checking optimization'
        m.optimize()
        #mg.optimize()
        #ll_gpy_opt = mg.log_likelihood()
        #assert_array_almost_equal(ll_gpy_opt, m.log_likelihood(), decimal=2) 
print 'done tests.'


********************************************************************************
d = 1, active_dims = None
[ 14:34:00 ] GP INFO: initializing Y
[ 14:34:00 ] GP INFO: initializing inference method
[ 14:34:00 ] GP INFO: adding kernel and likelihood as parameters
[ 14:34:00 ] gp_grief.kern DEBUG: Initializing RBF kernel.
[ 14:34:00 ] gp_grief.models DEBUG: Initializing GPRegression model.
checking gradient
[ 14:34:00 ] gp_grief.models INFO: Gradient check passed.
checking initial log-likelihood
testing if alpha matches
[ 14:34:00 ] gp_grief.models DEBUG: Fitting; determining weight vector.
testing if predictions match...
[ 14:34:00 ] gp_grief.models DEBUG: Predicting model at new points.
... mean
... variance
checking optimization
[ 14:34:00 ] gp_grief.models DEBUG: Beginning MLE to optimize hyperparameters. grad_method=finite_difference chol
[ 14:34:01 ] gp_grief.models INFO: Function Evals: 37. Exit status: Converged

********************************************************************

## Combining kernels
Try to reconstruct the kernel
$$ k = (k_0 k_1 + k_2)k_3$$

(see `kernel.testing.ipynb` for covariance matrix testing of this same kernel)

In [4]:
# generate data
np.random.seed(1)
N = 100
d = 5
x = np.random.uniform(size=(N,d)) # generate dataset
y = rastrigin((x*2-1)*5.12, lin_term=1.)
xx = np.random.uniform(size=(N+1,d)) # generate test set
    
# first define the base kernels
kb_gpy = [GPy.kern.RBF(    d, lengthscale=0.5*i+0.5, variance=0.5*i+0.5, name='k%d' % i) 
          for i in range(4)]
kb_kml = [gp_grief.kern.RBF(d, lengthscale=0.5*i+0.5, variance=0.5*i+0.5, name='k%d' % i) 
          for i in range(4)]

# then combine the base kernels
k_gpy = ((kb_gpy[0] * kb_gpy[1]) + kb_gpy[2]) * kb_gpy[3]
k_kml = ((kb_kml[0] * kb_kml[1]) + kb_kml[2]) * kb_kml[3]

# check to ensure they give the same kernel covariance matrix (also done in kernel_testing)
# ... this just ensures that everything was set up correctly
assert_array_almost_equal( k_kml.cov(x), k_gpy.K(x) )

# construct the models
m = dict()
m['gpy'] = GPy.models.GPRegression(  x,y,k_gpy)
m['kml'] = gp_grief.models.GPRegression(x,y,k_kml)
m['gpy'].mul.sum.mul.k1.variance.fix()
m['gpy'].mul.k3.variance.fix()
for key in m:
    print m[key]

print 'checking gradient'
assert m['kml'].checkgrad()

print 'checking initial log-likelihood'
assert_array_almost_equal(m['gpy'].log_likelihood(),m['kml'].log_likelihood(), decimal=3) 

print 'testing if predictions match...'
yyh = dict()
yyh['gpy'] = m['gpy'].predict(xx, full_cov=True)
yyh['kml'] = m['kml'].predict(xx, compute_var='full')
print '... mean'
assert_array_almost_equal(*[yyh[key][0] for key in m], decimal=2) 
print '... variance'
assert_array_almost_equal(*[yyh[key][1] for key in m], decimal=2) 

print 'testing if alpha matches'
assert_array_almost_equal(m['gpy'].posterior.woodbury_vector, m['kml']._alpha, decimal=3)
print 'done tests.'

print 'checking final log-likelihood after optimizing'
for key in m:
    m[key].optimize()
assert_array_almost_equal(m['gpy'].log_likelihood(),m['kml'].log_likelihood(), decimal=-1) 
print 'done tests.'

[ 14:34:02 ] gp_grief.kern DEBUG: Initializing k0 kernel.
[ 14:34:02 ] gp_grief.kern DEBUG: Initializing k1 kernel.
[ 14:34:02 ] gp_grief.kern DEBUG: Initializing k2 kernel.
[ 14:34:02 ] gp_grief.kern DEBUG: Initializing k3 kernel.
[ 14:34:02 ] GP INFO: initializing Y
[ 14:34:02 ] GP INFO: initializing inference method
[ 14:34:02 ] GP INFO: adding kernel and likelihood as parameters
[ 14:34:02 ] gp_grief.models DEBUG: Initializing GPRegression model.

GPRegression Model
| Name      |   Value | Constraint   |
|-----------+---------+--------------|
| noise_var |       1 | +ve          |

k0 kernel
| Name        |   Value | Constraint   |
|-------------+---------+--------------|
| variance    |     0.5 | +ve          |
| lengthscale |     0.5 | +ve          |

mul with child:

k1 kernel
| Name        |   Value | Constraint   |
|-------------+---------+--------------|
| variance    |       1 | fixed        |
| lengthscale |       1 | +ve          |


add with child:

k2 kernel
| Name      

 /home/trefor/Documents/libraries/GPy/GPy/kern/src/stationary.py:160: RuntimeWarning:overflow encountered in divide
 /home/trefor/Documents/libraries/GPy/GPy/kern/src/rbf.py:35: RuntimeWarning:overflow encountered in square
 /home/trefor/Documents/libraries/GPy/GPy/kern/src/rbf.py:38: RuntimeWarning:invalid value encountered in multiply
